# 数据清洗

In [1]:
%matplotlib inline
%matplotlib notebook
%load_ext autoreload
%autoreload 2

import vectorbtpro as vbt
import numpy as np
import pandas as pd

from numba import njit
import talib

# 管道优化
from vectorbtpro.returns import nb as ret_nb
from vectorbtpro.portfolio import nb as pf_nb
from vectorbtpro.portfolio.enums import Direction

vbt.settings.set_theme('dark')

In [2]:
print(vbt.__version__)

1.12.0


# 读入数据

In [3]:
symbol = "EURUSD"
time_frames = ["M1", "M5", "H1", "H4"]
date_range = slice("2021-01-01", "2023-02-20")
from_path = "./data"
tf_strs = "_".join(time_frames)
hdf_fname = f"{from_path}/{symbol}_{tf_strs}_OHLCV_{date_range.start}_{date_range.stop}.h5"
hdf_fname

# symbol = "XAUUSD"
# time_frames = ["H1"]
# date_range = slice("2020-01-01", "2023-05-16")
# tf_strs = "_".join(time_frames)
# from_path = "./data"
# hdf_fname = f"{from_path}/{symbol}_{tf_strs}_OHLCV_{date_range.start}_{date_range.stop}.h5"
# hdf_fname

'./data/XAUUSD_H1_OHLCV_2020-01-01_2023-05-16.h5'

In [4]:
## Load m1 data
data = vbt.HDFData.fetch(hdf_fname)
data.wrapper.index #pandas doaesn't recognise the frequency because of missing timestamps

DatetimeIndex(['2020-01-02 01:00:00+00:00', '2020-01-02 02:00:00+00:00',
               '2020-01-02 03:00:00+00:00', '2020-01-02 04:00:00+00:00',
               '2020-01-02 05:00:00+00:00', '2020-01-02 06:00:00+00:00',
               '2020-01-02 07:00:00+00:00', '2020-01-02 08:00:00+00:00',
               '2020-01-02 09:00:00+00:00', '2020-01-02 10:00:00+00:00',
               ...
               '2023-05-16 09:00:00+00:00', '2023-05-16 10:00:00+00:00',
               '2023-05-16 11:00:00+00:00', '2023-05-16 12:00:00+00:00',
               '2023-05-16 13:00:00+00:00', '2023-05-16 14:00:00+00:00',
               '2023-05-16 15:00:00+00:00', '2023-05-16 16:00:00+00:00',
               '2023-05-16 17:00:00+00:00', '2023-05-16 18:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='time', length=19924, freq=None)

In [5]:
data.wrapper.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [6]:
data.get()

,Open,High,Low,Close,Volume
time,,,,,
2020-01-02 01:00:00+00:00,1517.97,1521.37,1517.95,1518.41,3598
2020-01-02 02:00:00+00:00,1518.45,1519.37,1518.14,1518.69,2403
2020-01-02 03:00:00+00:00,1518.69,1520.91,1517.23,1519.44,6110
2020-01-02 04:00:00+00:00,1519.44,1520.95,1518.44,1520.63,3475
2020-01-02 05:00:00+00:00,1520.63,1521.21,1519.86,1520.37,1345
...,...,...,...,...,...
2023-05-16 14:00:00+00:00,2008.21,2009.32,2005.15,2005.51,4706
2023-05-16 15:00:00+00:00,2005.51,2014.86,2001.18,2011.99,11212
2023-05-16 16:00:00+00:00,2011.98,2012.34,2007.07,2007.59,12721


In [24]:
data_m1, data_m5, data_h1, data_h4 = data.get(symbols="EURUSD_M1"), data.get(symbols="EURUSD_M5"), \
                                     data.get(symbols="EURUSD_H1"), data.get(symbols="EURUSD_H4")
data_m1 = data.get(symbols="EURUSD_M1")

KeyError: 'EURUSD_M1'

In [12]:
data_h1.index

DatetimeIndex(['2020-01-02 01:00:00+00:00', '2020-01-02 02:00:00+00:00',
               '2020-01-02 03:00:00+00:00', '2020-01-02 04:00:00+00:00',
               '2020-01-02 05:00:00+00:00', '2020-01-02 06:00:00+00:00',
               '2020-01-02 07:00:00+00:00', '2020-01-02 08:00:00+00:00',
               '2020-01-02 09:00:00+00:00', '2020-01-02 10:00:00+00:00',
               ...
               '2023-05-16 09:00:00+00:00', '2023-05-16 10:00:00+00:00',
               '2023-05-16 11:00:00+00:00', '2023-05-16 12:00:00+00:00',
               '2023-05-16 13:00:00+00:00', '2023-05-16 14:00:00+00:00',
               '2023-05-16 15:00:00+00:00', '2023-05-16 16:00:00+00:00',
               '2023-05-16 17:00:00+00:00', '2023-05-16 18:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='time', length=19924, freq=None)

# 数据清洗
* 数据不连续校验
* 空值问题
* 数据明显错误校验
    * high >= (open, close) >= low
* 多周期匹配 (resampling问题)
* dukascopy 数据源 与 tickmill 原生数据源不匹配问题

## 数据不连续校验

## 空值问题

### 各周期空值显示

In [13]:
"m1数据空值:", data_m1.isna().any(axis=1).sum(), data_m1.size, data_m1.index.min(), data_m1.index.max()

NameError: name 'data_m1' is not defined

In [14]:
"m5数据空值:", data_m5.isna().any(axis=1).sum(), data_m5.size, data_m1.index.min(), data_m1.index.max()

NameError: name 'data_m5' is not defined

In [15]:
"H1数据空值:", data_h1.isna().any(axis=1).sum(), data_h1.size, data_m1.index.min(), data_m1.index.max()

NameError: name 'data_m1' is not defined

In [16]:
"H4数据空值:", data_h4.isna().any(axis=1).sum(), data_h4.size, data_m1.index.min(), data_m1.index.max()

NameError: name 'data_h4' is not defined

In [17]:
data_m1

NameError: name 'data_m1' is not defined

### 空值问题处理

In [18]:
data_m1[data_m1.isna().any(axis=1)].index

NameError: name 'data_m1' is not defined

In [19]:
"1m数据空值检查"
from clean_data_utils import check_nan_index
check_nan_index(data_m1[data_m1.isna().any(axis=1)].index, freq="1T")

NameError: name 'data_m1' is not defined

In [20]:
# data_m1.loc["2023-02-14 01:54:30+00:00"]

In [21]:
"1m数据空值处理"
data_m1 = data_m1.dropna()
data_m1[data_m1.isna().any(axis=1)]

NameError: name 'data_m1' is not defined

In [22]:
"5m数据空值检查"
from clean_data_utils import check_nan_index
check_nan_index(data_m5[data_m5.isna().any(axis=1)].index, freq="1T")

NameError: name 'data_m5' is not defined

In [23]:
"5m数据空值处理"

data_m5 = data_m5.dropna()
data_m5[data_m5.isna().any(axis=1)]

NameError: name 'data_m5' is not defined

In [ ]:
"1H数据空值检查"
from clean_data_utils import check_nan_index
check_nan_index(data_h1[data_h1.isna().any(axis=1)].index, freq="1T")

In [ ]:
"1H数据空值处理"
data_h1 = data_h1.dropna()
data_h1[data_h1.isna().any(axis=1)]

In [ ]:
"4H数据空值检查"
from clean_data_utils import check_nan_index
check_nan_index(data_h4[data_h4.isna().any(axis=1)].index, freq="1T")

In [ ]:
"4H数据空值处理"
data_h4 = data_h4.dropna()
data_h4[data_h4.isna().any(axis=1)]

## 多周期匹配问题

### 最小周期为 `1M`, 从 `1M` 开始检查

## dukascopy m1 resampling 和 m5, H1, H4 的数据不一致 (待检查)

# 保存数据

In [ ]:
data_m1.columns

In [ ]:
cleaned_hdf_m1_name = "./data/EURUSD_M1_OHLCV_2021-01-01_2023-02-20_cleaned.h5"
cleaned_hdf_m5_name = "./data/EURUSD_M5_OHLCV_2021-01-01_2023-02-20_cleaned.h5"
cleaned_hdf_h1_name = "./data/EURUSD_H1_OHLCV_2021-01-01_2023-02-20_cleaned.h5"
cleaned_hdf_h4_name = "./data/EURUSD_H4_OHLCV_2021-01-01_2023-02-20_cleaned.h5"

vbt.Data.from_data(data_m1).to_hdf(cleaned_hdf_m1_name, key="EURUSD_M1_Cleaned")
vbt.Data.from_data(data_m5).to_hdf(cleaned_hdf_m5_name, key="EURUSD_M5_Cleaned")
vbt.Data.from_data(data_h1).to_hdf(cleaned_hdf_h1_name, key="EURUSD_H1_Cleaned")
vbt.Data.from_data(data_h4).to_hdf(cleaned_hdf_h4_name, key="EURUSD_H4_Cleaned")

In [ ]:
hdf_fname = "./data/EURUSD_M1_OHLCV_2021-01-01_2023-02-20_cleaned.h5"
data = vbt.HDFData.fetch(hdf_fname)
data.wrapper.index, data.wrapper.columns, data.symbols